<a href="https://colab.research.google.com/github/MK25BM/offline-DRL/blob/main/Github_Offline_DRL_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

Create a Gymnasium-compatible wrapper around simglucose (https://github.com/jxx123/simglucose) simulator instance. Generate some offline patient data using the simulator. Wrap the environment with Minari DataCollector.

Using the above, demonstrate Offline Deep Reinforcement Learning (DRL) and Off-Policy Evaluation (OPE) by first defining an OpenAI Gym-compatible environment, implementing a behavior policy to collect an offline dataset, then implementing and training an Offline DRL algorithm on this dataset. Subsequently, implement and apply Off-Policy Evaluation (OPE) methods to estimate the performance of the trained offline policy using only the collected data. Finally, visualize the results, and summarize the demonstration, highlighting key findings, challenges of offline RL, and the utility of OPE.

# Redo verbose pipeline


# Library imports

In [ ]:
import os
import sys
import logging
from typing import Tuple, Any

def setup_dependencies() -> None:
    """Install and import required packages."""
    print("Setting up dependencies...")

    # Aggressive clean uninstall and cache purge
    os.system('pip uninstall -y gym gymnasium numpy minari d3rlpy scipy scikit-learn 2>/dev/null')
    os.system('pip cache purge')

    # Install compatible versions
    os.system('pip install -q "numpy==1.23.5" --force-reinstall')
    os.system('pip install -q "scipy==1.9.3" --force-reinstall')
    os.system('pip install -q "scikit-learn==1.2.2" --force-reinstall')
    os.system('pip install -q d3rlpy minari gymnasium gym --force-reinstall')

    print("OK All dependencies installed\n")

def import_libraries() -> Tuple[Any, Any, Any, Any]:
    """Import required libraries after installation."""
    import gymnasium
    import gym
    import d3rlpy
    import minari

    print(f"OK All imports successful")
    print(f"   Minari version: {minari.__version__}\n")

    return gymnasium, gym, d3rlpy, minari

def setup_logging(verbose: bool = True) -> None:
    """Configure logging for the pipeline."""

    if verbose:
        # Keep INFO level for our custom messages
        logging.basicConfig(level=logging.INFO)
    else:
        # Suppress most logging
        logging.basicConfig(level=logging.WARNING)

    # Suppress d3rlpy's verbose logging
    logging.getLogger('d3rlpy').setLevel(logging.WARNING)
    logging.getLogger('minari').setLevel(logging.WARNING)

# Call setup_dependencies first to ensure packages are installed
setup_dependencies()

# Then import the libraries
gymnasium, gym, d3rlpy, minari = import_libraries()

# MOCK T1D ENVIRONMENT

In [ ]:
import os
import sys
# Aggressive clean uninstall and cache purge to resolve numpy compatibility issues
os.system('pip uninstall -y gym gymnasium numpy minari d3rlpy scipy scikit-learn 2>/dev/null')
os.system('pip cache purge')

# Install compatible versions forcefully
os.system('pip install -q "numpy==1.23.5" --force-reinstall')
os.system('pip install -q "scipy==1.9.3" --force-reinstall')
os.system('pip install -q "scikit-learn==1.2.2" --force-reinstall')
os.system('pip install -q d3rlpy minari gymnasium gym --force-reinstall')

import gymnasium
import numpy as np
from typing import Optional, Tuple, Dict, Any

In [ ]:
# ============================================================================
# MOCK T1D ENVIRONMENT
# ============================================================================

class MockT1DEnv:
    """Mock Type 1 Diabetes environment for offline RL training."""

    def __init__(self):
        """Initialize mock environment."""
        self.current_glucose = 120.0
        self.time_step = 0
        self.max_steps = 480

    def reset(self):
        """Reset the environment."""
        min_glucose = 100.0
        max_glucose = 150.0
        self.current_glucose = np.random.uniform(min_glucose, max_glucose)
        self.time_step = 0
        return self.current_glucose

    def step(self, action):
        """Step the environment."""
        action = float(action)
        baseline = 15.0
        mean_noise = 0.0
        std_noise = 5.0
        noise = np.random.normal(mean_noise, std_noise)
        factor = 0.5
        delta = (action - baseline) * factor + noise
        self.current_glucose = self.current_glucose + delta
        self.current_glucose = np.clip(self.current_glucose, 40.0, 300.0)

        self.time_step = self.time_step + 1
        done = self.time_step >= self.max_steps

        if self.current_glucose < 70.0:
            reward = -1.0
        elif self.current_glucose > 180.0:
            reward = -0.5
        else:
            reward = 1.0

        info = {'glucose': self.current_glucose}

        return self.current_glucose, reward, done, info

    def close(self):
        """Close environment."""
        pass


# ============================================================================
# SIMGLUCOSE ENVIRONMENT WRAPPER - DIRECT INSTANTIATION (Now wraps MockT1DEnv)
# ============================================================================

class SimglucoseGymEnv(gymnasium.Env):
    """
    Gymnasium-compatible wrapper for SimGlucose T1DSimEnv.
    Now directly instantiates MockT1DEnv due to simglucose dependency issues.
    """

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(
        self,
        patient_name: str = 'adolescent#001',
        seed: Optional[int] = None,
        render_mode: Optional[str] = None
    ):
        """Initialize the SimglucoseGymEnv."""
        super().__init__() # Removed seed=seed here

        self.render_mode = render_mode
        self.patient_name = patient_name
        self._episode_steps = 0
        self._max_episode_steps = 480
        self._episode_rewards = []
        self._last_obs = None

        # Use MockT1DEnv instead of T1DSimEnv due to dependency conflicts
        self.env = MockT1DEnv()
        print(f"OK Successfully initialized MockT1DEnv (instead of T1DSimEnv due to compatibility issues).")

        self.action_space = gymnasium.spaces.Box(
            low=np.float32(0.0),
            high=np.float32(30.0),
            shape=(1,),
            dtype=np.float32
        )

        self.observation_space = gymnasium.spaces.Box(
            low=np.float32(0.0),
            high=np.float32(1000.0),
            shape=(1,),
            dtype=np.float32
        )

        # Ensure reproducibility for internal random operations if seed is provided
        if seed is not None:
            np.random.seed(seed)

    def step(self, action: np.ndarray) -> Tuple[np.ndarray, float, bool, bool, dict]:
        """Perform one step in the environment."""
        if isinstance(action, np.ndarray):
            scalar_action = float(action[0]) if action.size == 1 else float(action)
        else:
            scalar_action = float(action)

        try:
            observation, reward, done, info = self.env.step(scalar_action)

        except Exception as e:
            print(f"ERROR in step: {e}")
            import traceback
            traceback.print_exc()
            raise

        self._episode_steps += 1
        self._episode_rewards.append(float(reward))

        if observation is None:
            observation = self._last_obs if self._last_obs is not None else np.array([0.0], dtype=np.float32)
        else:
            if not isinstance(observation, np.ndarray):
                observation = np.array([float(observation)], dtype=np.float32)
            else:
                if observation.ndim == 0:
                    observation = np.array([float(observation)], dtype=np.float32)
                elif observation.shape == (1,):
                    observation = observation.astype(np.float32)
                else:
                    observation = np.array([float(observation.flat[0])], dtype=np.float32)
            self._last_obs = observation.copy()

        truncated = self._episode_steps >= self._max_episode_steps

        return observation, float(reward), bool(done), truncated, info

    def reset(
        self,
        seed: Optional[int] = None,
        options: Optional[dict] = None
    ) -> Tuple[np.ndarray, dict]:
        """Reset the environment."""
        super().reset(seed=seed)

        try:
            # If a seed is provided to reset, ensure MockT1DEnv uses it for reproducibility
            if seed is not None:
                np.random.seed(seed)
            observation = self.env.reset()

        except Exception as e:
            print(f"ERROR in reset: {e}")
            import traceback
            traceback.print_exc()
            raise

        if observation is None:
            observation = np.array([0.0], dtype=np.float32)
        else:
            if not isinstance(observation, np.ndarray):
                observation = np.array([float(observation)], dtype=np.float32)
            else:
                if observation.ndim == 0:
                    observation = np.array([float(observation)], dtype=np.float32)
                elif observation.shape == (1,):
                    observation = observation.astype(np.float32)
                else:
                    observation = np.array([float(observation.flat[0])], dtype=np.float32)

        self._last_obs = observation.copy()
        self._episode_steps = 0
        self._episode_rewards = []

        return observation, {}

    def render(self) -> Optional[Any]:
        """Render the environment (if applicable)."""
        return None

    def close(self) -> None:
        """Close the environment and cleanup resources."""
        if hasattr(self, 'env'):
            self.env.close()


# ============================================================================
# ENVIRONMENT SETUP AND TESTING
# ============================================================================

def setup_simglucose_environment(
    patient_name: str = 'adolescent#001',
    seed: int = 42
) -> SimglucoseGymEnv:
    """Setup and initialize a SimGlucose environment."""
    print("Initializing SimglucoseGymEnv...")
    env = SimglucoseGymEnv(patient_name=patient_name, seed=seed)
    print("OK SimglucoseGymEnv initialized successfully!")
    return env


def test_environment(env: SimglucoseGymEnv, n_steps: int = 5) -> None:
    """Test the environment with random actions."""
    print("\n" + "=" * 80)
    print("Testing Environment with Random Actions")
    print("=" * 80)

    obs, info = env.reset(seed=42)
    print(f"\nOK Reset successful!")
    print(f"Initial Observation: {obs}")

    print("\n--- Testing Steps ---")
    episode_rewards = []

    for i in range(n_steps):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        episode_rewards.append(reward)

        print(f"Step {i + 1}: obs={obs[0]:.2f}, reward={reward:.4f}")

        if terminated or truncated:
            print("  Episode ended!")
            break

    print(f"\n--- Episode Summary ---")
    print(f"  Total Steps: {len(episode_rewards)}")
    print(f"  Total Return: {sum(episode_rewards):.4f}")
    print(f"  Average Reward: {np.mean(episode_rewards):.4f}")
    print("\n" + "=" * 80)


def define_behavior_policy(observation: np.ndarray, env: SimglucoseGymEnv) -> np.ndarray:
    """Simple random behavior policy for data collection."""
    return env.action_space.sample()

# Data Collection

In [ ]:
# ============================================================================
# DATA COLLECTION
# ============================================================================

def setup_data_collection(
    env: SimglucoseGymEnv,
    num_episodes: int = 10,
    max_steps_per_episode: int = 480,
    dataset_name: str = None  # Add this parameter
) -> Tuple[str, DataCollector]: # Modified to return DataCollector
    """Setup and prepare data collection with Minari DataCollector."""
    print("\n" + "=" * 80)
    print("Setting up Minari Data Collection")
    print("=" * 80)

    # Use provided dataset_name or generate one
    if dataset_name is None:
        timestamp = time.strftime("%Y%m%d%H%M%S")
        dataset_name = f'simglucose/adolescent/random-v{timestamp}'

    print(f"\nCreating DataCollector for dataset '{dataset_name}'...")
    # Initialize Minari DataCollector without dataset_id in constructor
    data_collector = DataCollector(env, record_infos=True)
    print("OK DataCollector created successfully!")

    print(f"\nData Collection Parameters:")
    print(f"  - Dataset Name: {dataset_name}")
    print(f"  - Number of Episodes: {num_episodes}")
    print(f"  - Max Steps per Episode: {max_steps_per_episode}")

    return dataset_name, data_collector


def collect_data_simple(
    env:   SimglucoseGymEnv,
    policy:  Callable,
    num_episodes:  int = 10,
    max_steps_per_episode: int = 480,
    dataset_name: str = 'simglucose-adolescent-random-v0',
    verbose: bool = True
):
    """Collect trajectory data and return as dictionary."""
    if verbose:
        print(f"\nStarting data collection for {num_episodes} episodes...")
        start_time = time.time()

    episodes_data = []

    for episode_num in range(num_episodes):
        observations = []
        actions = []
        rewards = []
        terminations = []
        truncations = []

        obs, _ = env.reset()

        for step in range(max_steps_per_episode):
            observations.append(obs.  copy())
            action = policy(obs, env)
            actions.append(action)

            obs, reward, terminated, truncated, info = env.step(action)
            rewards.append(reward)
            terminations.append(terminated)
            truncations.append(truncated)

            if terminated or truncated:
                break

        episode_dict = {
            'observations': np.array(observations),
            'actions': np.array(actions),
            # ✅ Reshape rewards to 2D:  (n_steps, 1)
            'rewards': np.array(rewards, dtype=np.float32).reshape(-1, 1),
            'terminations': np.array(terminations),
            'truncations': np.  array(truncations),
        }
        episodes_data.append(episode_dict)

    elapsed_time = time.time() - start_time if verbose else None

    if verbose:
        print(f"OK Data collection complete! (took {elapsed_time:.2f}s)")
        print(f"Collected {len(episodes_data)} episodes")
        print(f"   Total transitions: {sum(len(ep['rewards']) for ep in episodes_data)}")

    return episodes_data


def collect_data_and_save(
    num_episodes: int = 10,
    max_steps_per_episode: int = 480,
    patient_name: str = 'adolescent#001',
    policy_type: str = 'random'
):
    """Complete SimGlucose environment setup and data collection pipeline."""
    print("=" * 80)
    print("SimGlucose Environment Setup and Data Collection")
    print("=" * 80)

    env = setup_simglucose_environment(patient_name=patient_name, seed=42)
    test_environment(env, n_steps=5)

    policy = define_behavior_policy
    print("\nOK Using random policy")

    episodes_data = collect_data_simple(
        env=env,
        policy=policy,
        num_episodes=num_episodes,
        max_steps_per_episode=max_steps_per_episode,
        dataset_name=None,
        verbose=True
    )

    env.close()

    print("\n" + "=" * 80)
    print("OK Data Collection Pipeline Complete!")
    print("=" * 80)

    return episodes_data

# DATASET MANAGEMENT

In [ ]:
# ============================================================================
# MINARI DATASET MANAGEMENT & REPLAY BUFFER MANAGEMENT
# ============================================================================

class MinariDatasetLoader:
    """Handler for loading and managing Minari datasets."""

    @staticmethod
    def load_dataset(dataset_name: str = 'simglucose-adolescent-random-v0') -> MinariDataset:
        """Load a Minari dataset."""
        print(f"\n{'='*80}")
        print(f"Loading Minari Dataset:  {dataset_name}")
        print(f"{'='*80}")

        try:
            # Try loading the exact dataset_name passed, which might be a pre-existing one
            # or a freshly generated unique one from collect_data_and_save.
            dataset = load_dataset(dataset_name, download=False)
            print(f"OK Dataset loaded successfully!")
            print(f"  - Total episodes: {len(dataset.episodes)}")
            total_transitions = sum(ep.transition_count for ep in dataset.episodes)
            print(f"  - Total transitions: {total_transitions}")
            return dataset

        except FileNotFoundError:
            print(f"Dataset '{dataset_name}' not found locally. Collecting new data...")
            # If not found, generate a unique name and collect new data
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            unique_dataset_name = f'simglucose-adolescent-random-v0-{timestamp}'
            dataset = collect_data_and_save(
                num_episodes=10,
                max_steps_per_episode=480,
                patient_name='adolescent#001',
                policy_type='random'
            )
            return dataset

    @staticmethod
    def dataset_statistics(dataset: MinariDataset) -> Dict[str, Any]:
        """Compute statistics about the dataset."""
        episodes = dataset.episodes
        episode_returns = []
        episode_lengths = []
        rewards_list = []

        for episode in episodes:
            episode_reward = 0.0
            for i in range(episode.transition_count):
                transition = episode[i]
                episode_reward += transition.reward
                rewards_list.append(transition.reward)
            episode_returns.append(episode_reward)
            episode_lengths.append(episode.transition_count)

        stats = {
            'num_episodes': len(episodes),
            'total_transitions': sum(episode_lengths),
            'mean_episode_return': float(np.mean(episode_returns)),
            'std_episode_return': float(np.std(episode_returns)),
            'max_episode_return': float(np.max(episode_returns)),
            'min_episode_return': float(np.min(episode_returns)),
            'mean_episode_length': float(np.mean(episode_lengths)),
            'mean_reward': float(np.mean(rewards_list)),
            'std_reward': float(np.std(rewards_list)),
        }
        return stats

    @staticmethod
    def print_statistics(stats: Dict[str, Any]) -> None:
        """Print dataset statistics."""
        print(f"\nDataset Statistics:")
        print(f"  Episodes: {stats['num_episodes']}")
        print(f"  Total Transitions: {stats['total_transitions']}")
        print(f"  Mean Episode Return: {stats['mean_episode_return']:.4f} +/- {stats['std_episode_return']:.4f}")
        print(f"  Mean Episode Length: {stats['mean_episode_length']:.1f}")
        print(f"  Mean Reward: {stats['mean_reward']:.4f} +/- {stats['std_reward']:.4f}")


# ============================================================================
# REPLAY BUFFER MANAGEMENT
# ============================================================================

class ReplayBufferManager:
    """Handler for converting Minari datasets to d3rlpy ReplayBuffer."""

    @staticmethod
    def create_replay_buffer(dataset: MinariDataset) -> Any:
        """Convert Minari dataset to d3rlpy ReplayBuffer."""
        print(f"\n{'='*80}")
        print("Converting Minari Dataset to d3rlpy ReplayBuffer")
        print(f"{'='*80}")

        try:
            replay_buffer = d3rlpy.dataset.create_replay_buffer(
                episodes=dataset.episodes
            )
            print(f"OK ReplayBuffer created successfully!")
            print(f"  - Size: {len(replay_buffer)} transitions")
            return replay_buffer
        except Exception as e:
            print(f"ERROR creating replay buffer: {e}")
            raise

# OFFLINE RL ALGORITHM CONFIGURATION & TRAINING


In [ ]:
# ============================================================================
# OFFLINE RL ALGORITHM CONFIGURATION
# ============================================================================

class OfflineRLAlgorithm:
    """Factory for creating offline RL algorithms."""

    ALGORITHM_CONFIGS = {
        'cql': {
            'name': 'Conservative Q-Learning (CQL)',
            'description': 'Best for general offline RL',
            'config_class': d3rlpy.algos.CQLConfig,
            'params': {
                'actor_learning_rate': 1e-4,
                'critic_learning_rate': 3e-4,
                'batch_size': 256,
                'gamma': 0.99,
                'tau': 5e-3,
                'alpha_learning_rate': 1e-4,
                'conservative_weight': 10.0,
                'n_action_samples': 10,
            }
        },
        'iql': {
            'name':  'Implicit Q-Learning (IQL)',
            'description': 'Avoids querying unseen actions',
            'config_class':  d3rlpy.algos.IQLConfig,
            'params': {
                'actor_learning_rate': 3e-4,
                'critic_learning_rate': 3e-4,
                'batch_size': 256,
                'gamma': 0.99,
                'tau': 5e-3,
                'expectile':  0.7,
                'weight_temp': 3.0,
                'max_weight': 100.0,
            }
        },
        'bc': {
            'name': 'Behavioral Cloning (BC)',
            'description': 'Simple imitation learning baseline',
            'config_class':  d3rlpy.algos.BCConfig,
            'params': {
                'learning_rate': 1e-4,
                'batch_size': 256,
            }
        },
    }

    @classmethod
    def list_algorithms(cls) -> None:
        """List available algorithms."""
        print(f"\n{'='*80}")
        print("Available Offline RL Algorithms")
        print(f"{'='*80}")
        for algo_type, config in cls.ALGORITHM_CONFIGS.items():
            print(f"\n  {algo_type.upper()}:\n    Name: {config['name']}\n    Description: {config['description']}")

    @classmethod
    def create(cls, algo_type: str = 'cql', device: str = 'cpu:0', **custom_params):
        """Create an offline RL algorithm."""
        algo_type = algo_type.lower()

        if algo_type not in cls.ALGORITHM_CONFIGS:
            raise ValueError(f"Unknown algorithm:  {algo_type}")

        config_info = cls.ALGORITHM_CONFIGS[algo_type]
        params = config_info['params'].copy()
        params.update(custom_params)

        print(f"\n{'='*80}")
        print(f"Creating {config_info['name']}")
        print(f"{'='*80}")
        print(f"Device: {device}")
        print(f"\nHyperparameters:")
        for key, value in params.items():
            print(f"  - {key}: {value}")

        config = config_info['config_class'](**params)
        algo = config.create(device=device)

        print(f"OK {config_info['name']} created successfully!")
        return algo


# ============================================================================
# TRAINING
# ============================================================================

class OfflineRLTrainer:
    """Trainer for offline RL algorithms."""

    @staticmethod
    def train(
        algo,
        replay_buffer,
        n_steps:  int = 50000,
        save_interval: int = 5000,
        verbose: bool = True
    ) -> Dict[str, Any]:
        """Train offline RL algorithm."""
        print("\n" + "="*80)
        print("Starting Offline RL Training")
        print("="*80)
        print("Training Configuration:")
        print(f"  - Algorithm: {algo.__class__.__name__}")
        print(f"  - Total Steps: {n_steps}")
        # ✅ Use . size() instead of len()
        print(f"  - Replay Buffer Size: {replay_buffer.size()}")
        print()

        start_time = time.time()

        try:
            # Train the algorithm
            algo.fit(
                replay_buffer,
                n_steps=n_steps,
                show_progress=verbose
            )

            training_time = time.time() - start_time

            if verbose:
                print(f"\nOK Training complete!")
                print(f"  - Training Time: {training_time:.2f}s") # Fixed f-string format
                print(f"  - Steps per Second: {n_steps / training_time:.2f}")

            return {
                'n_steps': n_steps,
                'training_time': training_time,
                'steps_per_second': n_steps / training_time
            }

        except Exception as e:
            if verbose:
                print(f"ERROR during training: {e}")
            raise


# EVALUATION & MODEL PERSISTENCE

In [ ]:

# ============================================================================
# EVALUATION
# ============================================================================

class PolicyEvaluator:
    """Handler for evaluating trained policies."""

    @staticmethod
    def evaluate(
        algo,
        env,
        n_episodes: int = 5,
        max_steps:  Optional[int] = None,
        verbose: bool = True
    ) -> Dict[str, float]:
        """Evaluate a trained policy."""
        if verbose:
            print(f"\n{'='*80}")
            print(f"Evaluating Policy ({n_episodes} episodes)")
            print(f"{'='*80}")

        episode_returns = []
        episode_lengths = []

        for episode_num in range(n_episodes):
            obs, _ = env.reset()
            total_return = 0.0
            steps = 0

            done = False
            while not done:
                action = algo.predict(np.expand_dims(obs, axis=0))[0]
                obs, reward, terminated, truncated, _ = env.step(action)
                total_return += reward
                steps += 1

                done = terminated or truncated
                if max_steps and steps >= max_steps:
                    done = True

            episode_returns.append(total_return)
            episode_lengths.append(steps)

            if verbose:
                print(f"  Episode {episode_num + 1:3d}: Return = {total_return:8.2f}, Steps = {steps:3d}")

        mean_return = float(np.mean(episode_returns))
        std_return = float(np.std(episode_returns))
        mean_length = float(np.mean(episode_lengths))

        stats = {
            'mean_return': mean_return,
            'std_return': std_return,
            'max_return': float(np.max(episode_returns)),
            'min_return': float(np.min(episode_returns)),
            'mean_episode_length': mean_length,
        }

        if verbose:
            print(f"\nOK Evaluation Complete!")
            print(f"  - Mean Return: {mean_return:.4f} +/- {std_return:.4f}")

        return stats


# ============================================================================
# MODEL PERSISTENCE
# ============================================================================

class ModelManager:
    """Handler for saving and loading models."""

    @staticmethod
    def save_model(algo, save_path: str = "./offline_rl_model") -> None:
        """Save a trained model."""
        print(f"\nSaving model to {save_path}...")
        algo.save_model(save_path)
        print(f"OK Model saved successfully!")

    @staticmethod
    def load_model(
        algo_type: str,
        save_path: str = "./offline_rl_model",
        device: str = "cpu:0"
    ):
        """Load a trained model."""
        print(f"\nLoading model from {save_path}...")
        config_class = OfflineRLAlgorithm.ALGORITHM_CONFIGS[algo_type.lower()]['config_class']
        algo = config_class().create(device=device)
        algo.load_model(save_path)
        print(f"OK Model loaded successfully!")
        return algo

# COMPLETE PIPELINE & MAIN EXECUTION

In [ ]:
# ============================================================================
# COMPLETE PIPELINE
# ============================================================================

def main(
    episodes_data:      List[Dict],  # Accept episodes data directly
    algo_type:  str = 'cql',
    n_steps:  int = 100000,
    n_eval_episodes: int = 5,
    device:  str = 'cpu:  0',
    save_model_flag:  bool = True
) -> Dict[str, Any]:
    """Complete offline RL pipeline."""
    print("\n" + "="*80)
    print("OFFLINE DEEP REINFORCEMENT LEARNING PIPELINE")
    print("="*80)

    results = {}

    OfflineRLAlgorithm.  list_algorithms()

    # Create replay buffer directly from episodes
    try:
        # ✅ Convert dict episodes to d3rlpy Episode objects
        from d3rlpy.dataset import Episode

        d3rlpy_episodes = []
        for episode_dict in episodes_data:
            # ✅ Episode requires (observations, actions, rewards, terminated:   bool)
            # terminated should be True if episode ended, False otherwise
            episode = Episode(
                observations=episode_dict['observations'],
                actions=episode_dict['actions'],
                rewards=episode_dict['rewards'],
                terminated=True  # ✅ Mark as terminated (episode finished)
            )
            d3rlpy_episodes.append(episode)

        # Now create replay buffer with proper Episode objects
        replay_buffer = d3rlpy.dataset.create_fifo_replay_buffer(
            episodes=d3rlpy_episodes,
            limit=1000000  # 1 million transitions max
        )
        print(f"\nOK ReplayBuffer created successfully!")
        # ✅ Use size() method or get total_transitions
        buffer_size = replay_buffer.size()
        print(f"  - Size: {buffer_size} transitions")
        results['replay_buffer_size'] = buffer_size
    except Exception as e:
        print(f"ERROR creating replay buffer: {e}")
        import traceback
        traceback.print_exc()
        raise

    algo = OfflineRLAlgorithm.  create(algo_type=algo_type, device=device)
    results['algorithm'] = algo_type

    training_stats = OfflineRLTrainer. train(
        algo=algo,
        replay_buffer=replay_buffer,
        n_steps=n_steps,
        save_interval=max(1000, n_steps // 10),
        verbose=True
    )
    results['training_stats'] = training_stats

    try:
        from gymnasium.  wrappers import TimeLimit
        eval_env = TimeLimit(
            SimglucoseGymEnv(patient_name='adolescent#001', seed=42),
            max_episode_steps=480
        )

        eval_stats = PolicyEvaluator.evaluate(
            algo=algo,
            env=eval_env,
            n_episodes=n_eval_episodes,
            max_steps=480,
            verbose=True
        )
        results['evaluation_stats'] = eval_stats
        eval_env.close()

    except Exception as e:
        print(f"\nWARNING Error during evaluation: {e}")

    if save_model_flag:
        model_path = f"./offline_rl_{algo_type}_model"
        ModelManager.save_model(algo, save_path=model_path)
        results['model_path'] = model_path

    print(f"\n{'='*80}")
    print("TRAINING SUMMARY")
    print(f"{'='*80}")
    print(f"OK Pipeline Complete!")
    print(f"\nResults:")
    print(f"  - Algorithm: {results['algorithm']}")
    print(f"  - Training Steps: {results['training_stats']['n_steps']}")
    print(f"  - Training Time: {results['training_stats']['training_time']:.2f}s")
    print(f"  - Replay Buffer Size: {results['replay_buffer_size']}")

    if 'evaluation_stats' in results:
        eval_stats = results['evaluation_stats']
        print(f"  - Evaluation Mean Return: {eval_stats['mean_return']:.4f} +/- {eval_stats['std_return']:.4f}")

    if 'model_path' in results:
        print(f"  - Model Saved:   {results['model_path']}")

    return results


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("=" * 70)
    print("PHASE 1: ENVIRONMENT VERIFICATION")
    print("=" * 70)
    # ...  Phase 1 code unchanged ...

    print("\n" + "=" * 80)
    print("PHASE 2: DATA COLLECTION")
    print("=" * 80)

    episodes_data = None
    try:
        episodes_data = collect_data_and_save(
            num_episodes=10,
            max_steps_per_episode=480,
            patient_name='adolescent#001',
            policy_type='random'
        )
        print(f"OK Collected {len(episodes_data)} episodes")
    except Exception as e:
        print(f"ERROR during data collection: {e}")
        import traceback
        traceback.print_exc()

    print("\n" + "=" * 80)
    print("PHASE 3: OFFLINE RL TRAINING")
    print("=" * 80)

    try:
        if episodes_data is not None:
            results = main(
                episodes_data=episodes_data,
                algo_type='cql',
                n_steps=50000,
                n_eval_episodes=3,
                device='cpu:0',
                save_model_flag=True
            )
            print("\n" + "=" * 80)
            print("OK TRAINING COMPLETE!")
            print("=" * 80)
        else:
            print("WARNING: No episodes collected.  Skipping training.")
    except Exception as e:
        print(f"ERROR during training: {e}")
        import traceback
        traceback.print_exc()

    print("\n" + "=" * 80)
    print("OK COMPLETE OFFLINE DRL PIPELINE FINISHED!")
    print("=" * 80)